#### `load_img`

In [13]:
from PIL import Image
import io
from pathlib import Path

_PIL_INTERPOLATION_METHODS = {
        'nearest': Image.NEAREST,
        'bilinear': Image.BILINEAR,
        'bicubic': Image.BICUBIC,
    }

def load_img_seve(path,target_size=(100,100), interpolation='nearest'):
    # read the path/image in bytes, and store it in img
    if isinstance(path, io.BytesIO):
        img = Image.open(path)
    elif isinstance(path, (Path, bytes, str)):
        if isinstance(path, Path):
            path = str(path.resolve())
        with open(path, 'rb') as f:
            img = Image.open(io.BytesIO(f.read()))
    else:
        raise TypeError('path should be path-like or io.BytesIO'
                        ', not {}'.format(type(path)))
    
    # resize the image
    if target_size is not None:
        width_height_tuple = (target_size[1], target_size[0])
        resample = _PIL_INTERPOLATION_METHODS[interpolation]
        img = img.resize(width_height_tuple, resample)
    
    return img

#### `img_to_array`

In [4]:
import numpy as np

def img_to_array_seve(img, dtype='float32'):
    x = np.asarray(img, dtype=dtype)
    if len(x.shape) == 2:
            x = x.reshape((x.shape[0], x.shape[1], 1))
    return x

#### Test

In [6]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array

In [14]:
test_image_path = '../raw_data/test-images/0.jpg'
dim=(100,100)

%timeit load_img(test_image_path, target_size=dim)
%timeit load_img_seve(test_image_path, target_size=dim)

20.7 ms ± 233 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
20.7 ms ± 253 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [15]:
img = load_img_seve(test_image_path,target_size=dim)

%timeit img_to_array(img)
%timeit img_to_array_seve(img)

54.7 µs ± 755 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
25.9 µs ± 254 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


#### avoid matplotlib as well (updating `jpg_to_array`)

In [1]:
from PIL import Image
import requests

test_link = 'https://www.wga.hu/art/c/cortona/2/3giove4.jpg'

im = Image.open(requests.get(test_link, stream=True).raw)

In [2]:
type(im)

PIL.JpegImagePlugin.JpegImageFile

In [6]:
from vincentvanbot.preprocessing.utils import img_to_array, jpg_to_array
import numpy as np

np.all(img_to_array(im) == jpg_to_array(test_link))

True